## Step 1 - Data Engineering

In [178]:
# Import the Pandas library
import pandas as pd
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

Base = declarative_base()

In [179]:
#create an engine
engine = create_engine('sqlite:///hawaii.sqlite', echo=True)
# Create a connection to the engine called `conn`
conn = engine.connect()

2018-04-07 03:34:39,477 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-07 03:34:39,480 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:34:39,482 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-07 03:34:39,484 INFO sqlalchemy.engine.base.Engine ()


In [180]:
measurement_df= pd.read_csv('./Clean_hawaii_measurements.csv')
station_df= pd.read_csv('./Clean_hawaii_stations.csv')

In [181]:
measurement_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,NaN,73


In [182]:
station_df.head()

,Unnamed: 0,Unnamed: 0.1,station,name,latitude,longitude,elevation
0,0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [183]:
#Creating Measurement and Station classes
class Measurement(Base):
    __tablename__ = "measurement"
    id = Column(Integer, primary_key = True)
    station = Column(String(255))
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
class Station(Base):
    __tablename__ = "station"
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    location = Column(Float)

In [184]:
# Create the tables in the database
Base.metadata.create_all(engine)

2018-04-07 03:34:39,615 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-04-07 03:34:39,618 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:34:39,622 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("station")
2018-04-07 03:34:39,625 INFO sqlalchemy.engine.base.Engine ()


In [185]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
### BEGIN SOLUTION
measurement_data = measurements_df.to_dict(orient='records')
measurement_data[0]

{'date': '2010-01-01', 'prcp': 0.08, 'station': 'USC00519397', 'tobs': 65}

In [186]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
### BEGIN SOLUTION
station_data = stations_df.to_dict(orient='records')
station_data[0]

{'Unnamed: 0': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [187]:
# Use MetaData from SQLAlchemy to reflect the tables
### BEGIN SOLUTION
metadata = MetaData(bind=engine)
metadata.reflect()

2018-04-07 03:34:39,911 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-04-07 03:34:39,915 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:34:39,925 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-04-07 03:34:39,928 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:34:39,933 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2018-04-07 03:34:39,938 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:34:39,942 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("measurement")
2018-04-07 03:34:39,945 INFO sqlalchemy.engine.base.Engine ()
2018-04-07 03:34:39,948 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2018-04-07 03:34:39,951 INFO sqlalchemy.engin

In [188]:
# Save the reference to the `measurement`  and 'station' table as a variable called 'measurement_table' and 'station table'
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [195]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

2018-04-07 03:36:13,778 INFO sqlalchemy.engine.base.Engine select * from measurement limit 5
2018-04-07 03:36:13,781 INFO sqlalchemy.engine.base.Engine ()


[]

In [190]:
#measurements_df.to_sql('measurement', engine, if_exists = "append", index=False)
#stations_df.to_sql('station', engine, if_exists = "append", index=False)